In [0]:
storage_account_name = "gcommstorage"
source_container = "bronze"
des_container = "silver"

storage_key = "VkPxRSwlU7rmBuv1Hnwl0hlKdKmWOyb/2gxH6WwindyXEhd5M28r1MRcDWL0YcsFwHHfJXboMZNC+AStt262CQ=="

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_key
)
print("Success")

print("-" * 30)
print("Attempting to read from:")
print("-" * 30)

Success
------------------------------
Attempting to read from:
------------------------------


In [0]:
dataset =  f"abfs://{source_container}@{storage_account_name}.dfs.core.windows.net/"

raw_df = spark.read.format("json").option("inferSchema", "true").load(dataset)



In [0]:
from pyspark.sql.types import IntegerType, DecimalType, TimestampType
from pyspark.sql.functions import col


final_df = raw_df.select(
    col("UserID"),
    col("Age").cast(IntegerType()),
    col("Gender"),
    col("Country"),
    col("Device"),
    col("GameGenre").alias("Game_Genre"),
    col("AverageSessionLength").cast(DecimalType(10, 2)).alias("Average_Session_Length"),
    col("SpendingSegment").alias("Spending_Segment"),
    col("InAppPurchaseAmount").cast(DecimalType(10, 2)).alias("In_App_Purchase_Amount"),
    col("FirstPurchaseDaysAfterInstall").cast(DecimalType(10, 2)).alias("FirstPurchaseDaysAfterInstall"),
    col("PaymentMethod").alias("Payment_Method"),
    # Date Fix
    (col("LastPurchaseDate").cast("long") / 1000).cast(TimestampType()).alias("Last_Purchase_Date")
).withColumn(
    "Total_Purchases", 
    col("In_App_Purchase_Amount") + col("FirstPurchaseDaysAfterInstall")
).filter(col("UserID").isNotNull())

final_df = final_df.dropna(how='any', subset=[
    "UserID",
    "Age",
    "Gender",
    "Country",
    "Device",
    "Game_Genre",
    "Average_Session_Length",
    "Spending_Segment",
    "In_App_Purchase_Amount",
    "FirstPurchaseDaysAfterInstall",
    "Payment_Method",
    "Last_Purchase_Date",
    "Total_Purchases"
])
final_df = final_df.drop("UserID")
display(final_df)

Age,Gender,Country,Device,Game_Genre,Average_Session_Length,Spending_Segment,In_App_Purchase_Amount,FirstPurchaseDaysAfterInstall,Payment_Method,Last_Purchase_Date,Total_Purchases
49,Male,Norway,Android,Battle Royale,12.83,Minnow,11.40,28.00,Apple Pay,2025-03-19T00:00:00Z,39.40
15,Male,Switzerland,iOS,Action RPG,19.39,Minnow,6.37,18.00,Debit Card,2025-06-08T00:00:00Z,24.37
23,Male,China,Android,Fighting,8.87,Minnow,15.81,30.00,Apple Pay,2025-06-02T00:00:00Z,45.81
31,Male,Mexico,Android,Racing,19.56,Minnow,13.49,9.00,Debit Card,2025-04-01T00:00:00Z,22.49
37,Female,India,Android,Battle Royale,15.23,Minnow,10.86,15.00,Paypal,2025-05-05T00:00:00Z,25.86
38,Female,China,Android,Fighting,25.97,Dolphin,150.51,28.00,Google Pay,2025-05-05T00:00:00Z,178.51
20,Female,Italy,Android,Simulation,10.98,Whale,608.21,24.00,Debit Card,2025-06-04T00:00:00Z,632.21
38,Male,Japan,Android,Action RPG,21.82,Minnow,11.52,1.00,Paypal,2025-04-08T00:00:00Z,12.52
46,Male,Russia,iOS,Battle Royale,16.36,Minnow,7.76,19.00,Google Pay,2025-02-03T00:00:00Z,26.76
29,Male,Switzerland,iOS,MMORPG,27.15,Minnow,13.44,12.00,Debit Card,2025-05-06T00:00:00Z,25.44


In [0]:
dataset =  f"abfs://{des_container}@{storage_account_name}.dfs.core.windows.net/"

final_df.write.mode("overwrite").format("parquet").save(dataset)